In [1]:
import os
from datetime import datetime

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
%cd /content
%ls

[Errno 2] No such file or directory: '/content'
/Users/macdonald/Desktop/MagicKnobRep/MagicKnob/python
__pycache__/                      myk_train.py
myk_data.py                       myk_train_conv.py*
myk_data_conv.py*                 runs/
myk_evaluate.py                   train.py
myk_evaluate_conv.py*             yk_training.ipynb
myk_loss.py                       yk_training_classic_conv1D.ipynb*
myk_models.py                     yk_training_classic_conv2D.ipynb*
myk_models_conv.py*


In [3]:
drive = False
if drive:
    path = "/content/drive/MyDrive/Progetto_MagicKnob/"

    # check python file folder
    assert os.path.exists(path + "yk_training_files"), f"Upload python files in {path}yk_training_files"
    %cd ./drive/MyDrive/Progetto_MagicKnob/yk_training_files

    # check data folder
    assert os.path.exists(path + "data"), f"Upload data files in {path}data"

In [4]:
#import myk_data
#import myk_models
import myk_loss
#import myk_train

import myk_data_conv
import myk_models_conv
import myk_train_conv
import torch
from torch.utils.data import DataLoader
#import myk_evaluate
import myk_evaluate_conv

In [5]:
from torch.utils.tensorboard import SummaryWriter

In [6]:
# used for the writing of example outputs
run_name="audio_conv2D"
# dataset : need an input and output folder in this folder
audio_folder = "../data/audio_conv2D"
assert os.path.exists(audio_folder), "Audio folder  not found. Looked for " + audio_folder
# used to render example output during training
test_file = "../data/guitar.wav"
assert os.path.exists(test_file), "Test file not found. Looked for " + test_file

In [7]:
# initialize net specs
lstm_hidden_size = 32
learning_rate = 5e-3
batch_size = 50
max_epochs = 10000

# create the logger for tensorboard
writer = SummaryWriter()

In [8]:
# Reading dataset
dataset = myk_data_conv.generate_dataset(audio_folder + "/input/", audio_folder + "/output/", frag_len_seconds=0.5)

# slitting ds into train val test
train_ds, val_ds, test_ds = myk_data_conv.get_train_valid_test_datasets(dataset)

generate_dataset:: Loaded frames from audio file 120
input tensor shape torch.Size([120, 22050, 1])


In [9]:
dataset[0]

(tensor([[-3.0518e-05],
         [-3.0518e-05],
         [ 0.0000e+00],
         ...,
         [ 0.0000e+00],
         [ 0.0000e+00],
         [ 0.0000e+00]]),
 tensor([[ 0.0000e+00],
         [-4.1723e-06],
         [ 8.3447e-06],
         ...,
         [ 6.1035e-04],
         [ 1.5259e-03],
         [ 1.2207e-03]]))

In [10]:
# test GPU, must be done after splitting
device = myk_train_conv.get_device()

cuda device not available/not selected


In [11]:
# create data loaders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, generator=torch.Generator(device=device))

In [12]:
model = myk_models_conv.SimpleConv2D().to(device)

In [13]:
# crate optimizer and loss function
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', factor=0.5, patience=5, verbose=True) non lo usava

loss_functions = myk_loss.LossWrapper()

# https://github.com/Alec-Wright/Automated-GuitarAmpModelling/blob/main/dist_model_recnet.py
# https://github.com/Alec-Wright/CoreAudioML/blob/bad9469f94a2fa63a50d70ff75f5eff2208ba03f/training.py

In [14]:
#%load_ext tensorboard
#%tensorboard --logdir logs

In [16]:
# training loop
lowest_val_loss = 0
best_loss = False

max_patience = 500
curr_patience = max_patience

# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")

os.mkdir(os.path.join(".", dt_string))

for epoch in range(max_epochs):
    #ep_loss = myk_train.train_epoch_interval(model, train_dl, loss_functions, optimiser, device=device)
    ep_loss = myk_train_conv.train_epoch_conv(model, train_dl, loss_functions, optimiser, device=device) #decomment per conv

    #ep_loss = myk_train.train_epoch(model, train_dl, loss_functions, optimiser, device=device)
    val_loss = myk_train_conv.compute_batch_loss(model, val_dl, loss_functions, device=device)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("Loss/train", ep_loss, epoch)

    # check if we have beaten our best loss to date
    if lowest_val_loss == 0:# first run
        lowest_val_loss = val_loss
    elif val_loss < lowest_val_loss:# new record
        lowest_val_loss = val_loss
        best_loss = True
    else: # no improvement
        best_loss = False
        curr_patience -= 1

    if best_loss: # save best model so far
        print("    Record loss - saving at epoch ", epoch)
        model.save_for_rtneural(f"{dt_string}/model.json")
        print(f"   epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss} ")
        curr_patience = max_patience
    if epoch % 50 == 0: # save an example processed audio file
        myk_evaluate_conv.run_file_through_model(model, test_file, audio_folder + "/" + run_name + str(epoch)+".wav")
        print(f"epoch {epoch}, train_loss {ep_loss}, val_loss {val_loss}")
    if curr_patience == 0:
        print("max patience reached, stopping")
        break

input shape: torch.Size([50, 22050, 1])


KeyboardInterrupt: 